NER on BERT implemented based on:

Reference: https://towardsdatascience.com/named-entity-recognition-with-bert-in-pytorch-a454405e0b6a 

Combining two datasets

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 182 kB 94.6 MB/s 
     |████████████████████████████████| 7.6 MB 84.4 MB/s 


In [3]:
import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 98.2 MB/s 
     |████████████████████████████████| 132 kB 90.6 MB/s 
     |████████████████████████████████| 127 kB 88.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
from datasets import load_dataset, load_metric, concatenate_datasets
datasets = load_dataset("Babelscape/wikineural")

Extracting data files #1:   0%|          | 0/2 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/2 [00:00<?, ?obj/s]

Extracting data files #6:   0%|          | 0/2 [00:00<?, ?obj/s]

Extracting data files #7:   0%|          | 0/2 [00:00<?, ?obj/s]

Extracting data files #15:   0%|          | 0/1 [00:00<?, ?obj/s]

...

Generating val_pt split: 0 examples [00:00, ? examples/s]

Generating val_it split: 0 examples [00:00, ? examples/s]

Generating train_en split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Babelscape___parquet/Babelscape--wikineural-579d1dc98d2a6b93/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/27 [00:00<?, ?it/s]

In [6]:
texts = []
for i in datasets["train_en"]:
    i['tokens'] = ' '.join(i['tokens'])
    texts.append(i['tokens'])

In [7]:
texts

['This division also contains the Ventana Wilderness , home to the California condor .',
 '" So here is the balance NBC has to consider : The Who , \' Animal Practice \' .',
 'It is a protest song that " creates a cinematic vista that tells of the singer \'s search for a literal and physical America that seems to have disappeared , along with the country \'s beauty and ideals " .',
 'This differs from approaches such as IP or Ethernet that use variable sized packets or frames .',
 'Since then , only Terry Bradshaw in 147 games , Joe Montana in 139 games , and Tom Brady in 131 games have reached 100 wins more quickly .',
 'Next is the Adolf-Hitler-Platz , a grand public square for rallies and such .',
 'Although the name " Minor 1000 " was retained , the changes were sufficient for the new model to be given its own ADO development number .',
 "But cutting ties with Taiwan was a sad and painful decision because of the friendship between Nicaragua and Taiwan 's people and government .",
 

In [8]:
tags = []
for i in datasets["train_en"]:
    tags.append(i['ner_tags'])

In [9]:
for i in tags:
  for j in range (len(i)):

    if str(i[j]) == '0':
      i[j] = 'O'
    if str(i[j]) == '1':
      i[j] = 'B-per'
    if str(i[j]) == '2':
      i[j] = 'I-per'
    if str(i[j]) == '3':
      i[j] = 'B-org'
    if str(i[j]) == '4':
      i[j] = 'I-org'
    if str(i[j]) == '5':
      i[j] = 'B-geo'
    if str(i[j]) == '6':
      i[j] = 'I-geo'
    if str(i[j]) == '7':
      i[j] = 'B-misc'
    if str(i[j]) == '8':
      i[j] = 'I-misc'
  #print(i)
    

In [10]:
tags2 =[]
for i in tags:
  j = ' '.join(i)
  b = ' '.join(str(j).split(','))
  tags2.append(b)


In [11]:
tags2

['O O O O O B-geo I-geo O O O O B-geo O O',
 'O O O O O O B-org O O O O B-misc I-misc O O B-misc I-misc O O',
 'O O O O O O O O O O O O O O O O O O O O O O O B-geo O O O O O O O O O O O O O O O O',
 'O O O O O O O O B-misc O O O O O O O O',
 'O O O O B-per I-per O O O O B-per I-per O O O O O B-per I-per O O O O O O O O O O',
 'O O O B-geo O O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O O O O O O O B-org O O O',
 'O O O O B-geo O O O O O O O O O O O B-geo O B-geo O O O O O',
 'O O O O O O O O O O O O B-geo O O O O O O O O O O O O',
 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-geo O O',
 'O O O O O O B-geo O O O O O O O O O O O B-geo O',
 'O O O O O O B-geo O O O O O O B-geo I-geo O B-geo O B-geo O B-geo O B-geo O B-geo O O B-geo I-geo O B-geo I-geo I-geo I-geo O O O O B-geo I-geo O O B-geo I-geo O O O O O O O O O O B-geo I-geo O B-geo O O B-geo O B-geo O O B-geo I-geo O O B-geo O',
 'O O O O O O O O O O O O O B-misc O O O B-misc O O O B-misc O O',
 'O O O O O O 

In [12]:
import csv

with open('ner2.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(texts, tags2))

In [13]:
import pandas as pd

df = pd.read_csv('ner2.csv')
df

,"This division also contains the Ventana Wilderness , home to the California condor .",O O O O O B-geo I-geo O O O O B-geo O O
0,""" So here is the balance NBC has to consider :...",O O O O O O B-org O O O O B-misc I-misc O O B-...
1,"It is a protest song that "" creates a cinemati...",O O O O O O O O O O O O O O O O O O O O O O O ...
2,This differs from approaches such as IP or Eth...,O O O O O O O O B-misc O O O O O O O O
3,"Since then , only Terry Bradshaw in 147 games ...",O O O O B-per I-per O O O O B-per I-per O O O ...
4,"Next is the Adolf-Hitler-Platz , a grand publi...",O O O B-geo O O O O O O O O O O
...,...,...
92714,"The couple had a son , David , and a daughter ...",O O O O O O B-per O O O O O B-per O
92715,"The Home Secretary , J. R. Clynes , was presen...",O O O O B-per I-per I-per O O O O O O O O
92716,"At the time of her birth , she was fourth in t...",O O O O O O O O O O O O O O O O O B-per I-per O
92717,The film was based on the Broadway play of the...,O O O O O O B-misc O O O O O O


In [14]:
headerList = ['text', 'labels']
  
# converting data frame to csv
df.to_csv("ner2.2.csv", header=headerList)
  
# display modified csv file
df2 = pd.read_csv("ner2.2.csv")
df2 = df2.drop(['Unnamed: 0'], axis=1)
df2.columns = ['text', 'labels']

In [15]:
df2

,text,labels
0,""" So here is the balance NBC has to consider :...",O O O O O O B-org O O O O B-misc I-misc O O B-...
1,"It is a protest song that "" creates a cinemati...",O O O O O O O O O O O O O O O O O O O O O O O ...
2,This differs from approaches such as IP or Eth...,O O O O O O O O B-misc O O O O O O O O
3,"Since then , only Terry Bradshaw in 147 games ...",O O O O B-per I-per O O O O B-per I-per O O O ...
4,"Next is the Adolf-Hitler-Platz , a grand publi...",O O O B-geo O O O O O O O O O O
...,...,...
92714,"The couple had a son , David , and a daughter ...",O O O O O O B-per O O O O O B-per O
92715,"The Home Secretary , J. R. Clynes , was presen...",O O O O B-per I-per I-per O O O O O O O O
92716,"At the time of her birth , she was fourth in t...",O O O O O O O O O O O O O O O O O B-per I-per O
92717,The film was based on the Broadway play of the...,O O O O O O B-misc O O O O O O


In [16]:
df = pd.read_csv('/content/drive/MyDrive/ner.csv')
df

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...
...,...,...
47954,Opposition leader Mir Hossein Mousavi has said...,O O O B-per I-per O O O O O O O O O O O O O O ...
47955,"On Thursday , Iranian state media published a ...",O B-tim O B-gpe O O O O O O O O B-org I-org O ...
47956,"Following Iran 's disputed June 12 elections ,...",O B-geo O O B-tim I-tim O O O O O O O O O O O ...
47957,"Since then , authorities have held public tria...",O O O O O O O O O O O O O O O O O O O O O


In [17]:
df.columns = ['text', 'labels']

In [18]:
#combining datasets
df = pd.concat([df, df2], ignore_index=True, axis=0)

In [19]:
df

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...
...,...,...
140673,"The couple had a son , David , and a daughter ...",O O O O O O B-per O O O O O B-per O
140674,"The Home Secretary , J. R. Clynes , was presen...",O O O O B-per I-per I-per O O O O O O O O
140675,"At the time of her birth , she was fourth in t...",O O O O O O O O O O O O O O O O O B-per I-per O
140676,The film was based on the Broadway play of the...,O O O O O O B-misc O O O O O O


In [20]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx
    #print(label_ids)
    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        #print(lb)
        txt = df['text'].values.tolist()
        #print(txt)
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [22]:
df = df[:140677]

labels = [i.split() for i in df['labels'].values.tolist()]
print(labels)
unique_labels = set()

for lb in labels:
        [unique_labels.add(i) for i in lb if i not in unique_labels]
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
print(ids_to_labels) 

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
 class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [24]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)
            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 0.005
EPOCHS = 3
BATCH_SIZE = 2

model = BertModel()
train_loop(model, df_train, df_val)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Epochs: 1 | Loss:  0.147 | Accuracy:  0.957 | Val_Loss:  0.093 | Accuracy:  0.972


100%|██████████| 56271/56271 [53:08<00:00, 17.65it/s]


Epochs: 2 | Loss:  0.087 | Accuracy:  0.973 | Val_Loss:  0.078 | Accuracy:  0.977


100%|██████████| 56271/56271 [53:15<00:00, 17.61it/s]


Epochs: 3 | Loss:  0.072 | Accuracy:  0.978 | Val_Loss:  0.075 | Accuracy:  0.978


In [25]:

def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

    
            for i in range(logits.shape[0]):

              logits_clean = logits[i][test_label[i] != -100]
              label_clean = test_label[i][test_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_test += acc
            

  
    val_accuracy = total_acc_test / len(df_test)
    print("val_accuracy", val_accuracy)
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')
    


evaluate(model, df_test)


val_accuracy tensor(0.9779, device='cuda:0')
Test Accuracy:  0.978


In [26]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()
      
    words = sentence.split()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]
    
    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]

    prediction_dict = dict(zip(words, prediction_label))
    

    #print(sentence)
    #print(prediction_label
    return prediction_dict



Dialogue System

In [30]:
travel_detail = ["Name", "Origin", "Destination", "Date of Travel", "Time of Travel", "Return Date", "Return Time" ]

frames = dict(zip(travel_detail, [None]*len(travel_detail)))
data =[frames]
print(frames)
print(data)

{'Name': None, 'Origin': None, 'Destination': None, 'Date of Travel': None, 'Time of Travel': None, 'Return Date': None, 'Return Time': None}
[{'Name': None, 'Origin': None, 'Destination': None, 'Date of Travel': None, 'Time of Travel': None, 'Return Date': None, 'Return Time': None}]


In [74]:

def chat():
  travel_detail = ["Name", "Origin", "Destination", "Date of Travel", "Time of Travel" ]
  frames = dict(zip(travel_detail, [None]*len(travel_detail)))
  data =[]
  name = ""
  dest = ""
  origin = ""
  DateofTravel = ""
  time = ""
  returnDate = ""
  returntime = ""
  print("Agent: Hi, do you want to book a flight ticket? [Reply Yes or No]")
  response = input()
  if response == "Yes" or response == "yes":
    print("Agent: Nice, let's get you started, What is your name?")
    name_response = input()
    nameResponse_label = evaluate_one_text(model, name_response)
    #print(nameResponse_label)
    for key, value in nameResponse_label.items():
      if "B-per" == value:
        name = key
      if "I-per" == value:
        name = name +" " +key
    frames["Name"] = name

    print("Agent: Where do you want to travel?")
    destination_response = input()
    destinationResponse_label = evaluate_one_text(model, destination_response)
    #print(destinationResponse_label)
    for key, value in destinationResponse_label.items():
      if "B-geo" == value:
        dest = key
      if "I-geo" == value:
        dest = dest +" " +key 
    frames["Destination"] = dest

    print("Agent: What is your place of origin?")
    origin_response = input()
    originResponse_label = evaluate_one_text(model, origin_response)
    #print(originResponse_label)
    for key, value in originResponse_label.items():
      if "B-geo" == value:
        origin = key
      if "I-geo" == value:    
        origin = origin +" " +key 
    frames["Origin"] = origin

    print("Agent: What day do you want to travel? [example: 12th December 2022]")
    DoT_response = input()
    DoTResponse_label = evaluate_one_text(model, DoT_response)
    #print(DoTResponse_label)
    for key, value in DoTResponse_label.items():
      if "B-tim" == value:
        DateofTravel = key
      if "I-tim" == value:    
        DateofTravel = DateofTravel +" " +key 
    frames["Date of Travel"] = DateofTravel
    frames["Date of Travel"] = DoT_response

    print("Agent: What time of the day do you want to travel? [example: morning, afternoon, evening]")
    time_response = input()
    timeResponse_label = evaluate_one_text(model, time_response)
    
    #print(timeResponse_label)
    for key, value in timeResponse_label.items():
      if "B-tim" == value:
        time = key
      if "I-tim" == value:    
        time = time +" " +key 
    frames["Time of Travel"] = time
    frames["Time of Travel"] = time_response

    print("Agent: Do you want a return ticket?[Reply Yes or No]")
    returnresponse = input()
    if returnresponse == "Yes" or returnresponse == "yes":
      print("Agent: When do you want to return?[example: 12th December 2022]")
      return_date = input()
      returnDateResponse_label = evaluate_one_text(model, return_date)
      #print(returnDateResponse_label)
      for key, value in returnDateResponse_label.items():
        if "B-tim" == value:
          returnDate = key
        if "I-tim" == value:    
          returnDate =  returnDate +" " +key 
      frames["Return Date"] =  returnDate
      frames["Return Date"] =  return_date
         
      print("Agent: What time of the day do you want your return flight? [example: morning, afternoon, evening] ")
      returnTime_response = input()
      returnTimeResponse_label = evaluate_one_text(model, returnTime_response)
      #print(returnTimeResponse_label)
      for key, value in returnTimeResponse_label.items():
        if "B-tim" == value:
          returntime = key
        if "I-tim" == value:    
          returntime = returntime +" " +key 
      frames["Return Time"] = returntime
      frames["Return Time"] = returnTime_response
    else:
      for key, value in frames.items():
        if "None" == value:
          print("Agent: More Information needed")
      
  
  for key, value in frames.items():
    if "None" == value:
      print("Agent: More Information needed")

  data.append(frames)
  print("Thank you")

  return frames, data




In [75]:
frames, data = chat()

Agent: Hi, do you want to book a flight ticket? [Reply Yes or No]
yes
Agent: Nice, lets get you started, What is your name?
My name is Nancy Drew
Agent: Where do you want to travel?
I want to travel to London.
Agent: What is your place of origin?
I am from Indianapolis. 
Agent: What day do you want to travel? [example: 12th December 2022]
12th June 2023
Agent: What time of the day do you want to travel? [example: morning, afternoon, evening]
morning
Agent: Do you want a return ticket?[Reply Yes or No]
Yes
Agent: When do you want to return?[example: 12th December 2022]
23rd June 2023
Agent: What time of the day do you want your return flight? [example: morning, afternoon, evening] 
afternoon
Thank you


In [76]:
print(data)

[{'Name': 'Nancy Drew', 'Origin': 'Indianapolis.', 'Destination': 'London.', 'Date of Travel': '12th June 2023', 'Time of Travel': 'morning', 'Return Date': '23rd June 2023', 'Return Time': 'afternoon'}]
